In [1]:
import numpy as np
import xarray as xr
import xrft as xf
import dask.array as dsar
import os.path as op
import scipy.fftpack as fft

from matplotlib import colors
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def synthetic_field(N=512, dL=1., amp=1e1, s=-3.):
    """
    Generate a synthetic series of size N by N 
    with a spectral slope of s.
    """

    k = fft.fftshift(fft.fftfreq(N, dL))
    l = fft.fftshift(fft.fftfreq(N, dL))
    kk, ll = np.meshgrid(k, l)
    K = np.sqrt(kk**2+ll**2)

    ########
    # amplitude
    ########
    r_kl = np.ma.masked_invalid(np.sqrt(amp*.5*(np.pi)**(-1)*K**(s-1.))).filled(0.)
    #r = np.ma.masked_invalid(np.abs(k)**(-slope/2.)).filled(0.)
    ########
    # phase
    ########
    phi = np.zeros((N, N))
    
    N_2 = int(N/2)
    phi_upper_right = 2.*np.pi*np.random.random((N_2-1, 
                                                 N_2-1)) - np.pi
    phi[N_2+1:,N_2+1:] = phi_upper_right.copy()
    phi[1:N_2, 1:N_2] = -phi_upper_right[::-1, ::-1].copy()


    phi_upper_left = 2.*np.pi*np.random.random((N_2-1, 
                                                N_2-1)) - np.pi
    phi[N_2+1:,1:N_2] = phi_upper_left.copy()
    phi[1:N_2, N_2+1:] = -phi_upper_left[::-1, ::-1].copy()


    phi_upper_middle = 2.*np.pi*np.random.random(N_2) - np.pi
    phi[N_2:, N_2] = phi_upper_middle.copy()
    phi[1:N_2, N_2] = -phi_upper_middle[1:][::-1].copy()


    phi_right_middle = 2.*np.pi*np.random.random(N_2-1) - np.pi
    phi[N_2, N_2+1:] = phi_right_middle.copy()
    phi[N_2, 1:N_2] = -phi_right_middle[::-1].copy()


    phi_edge_upperleft = 2.*np.pi*np.random.random(N_2) - np.pi
    phi[N_2:, 0] = phi_edge_upperleft.copy()
    phi[1:N_2, 0] = -phi_edge_upperleft[1:][::-1].copy()


    phi_bot_right = 2.*np.pi*np.random.random(N_2) - np.pi
    phi[0, N_2:] = phi_bot_right.copy()
    phi[0, 1:N_2] = -phi_bot_right[1:][::-1].copy()


    phi_corner_leftbot = 2.*np.pi*np.random.random() - np.pi


#     print(phi[N/2-1,N-1], phi[N/2+1,1])
#     print(phi[N/2+1,N/2+1], phi[N/2-1,N/2-1])


#     phi[:N/2, :] = -np.rot90(np.rot90(phi[N/2:, :]))
#     phi[:N/2, :] = -phi[N/2:, :][::-1,::-1]
#     i, j = 25, 40
#     print(phi[N/2+j,N/2+i], -phi[N/2-j,N/2-i])

    for i in range(1, N_2):
        for j in range(1, N_2):
            assert (phi[N_2+j, N_2+i] == -phi[N_2-j, N_2-i])
        
    for i in range(1, N_2):
        for j in range(1, N_2):
            assert (phi[N_2+j, N_2-i] == -phi[N_2-j, N_2+i])
        
    for i in range(1, N_2):
        assert (phi[N_2, N-i] == -phi[N_2, i])
        assert (phi[N-i, N_2] == -phi[i, N_2])
        assert (phi[N-i, 0] == -phi[i, 0])
        assert (phi[0, i] == -phi[0, N-i])
    #########
    # complex fourier amplitudes
    #########
    #a = r + 1j*th
    F_theta = r_kl * np.exp(1j * phi)
    
    # check that symmetry of FT is satisfied
    #np.testing.assert_almost_equal(a[1:N/2], a[-1:-N/2:-1].conj())

    theta = fft.ifft2(fft.ifftshift(F_theta))
    return np.real(theta) 

In [3]:
theta = synthetic_field(N=512, s=-3.)
print(theta.shape)
theta = xr.DataArray(theta, dims=['k', 'l'],
            coords={'k':range(512), 'l':range(512)})
# theta = dsar.from_array(theta, chunks=(32, 32))
theta

(512, 512)


/Users/uchidatakaya/anaconda/envs/xrft/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in power
  from ipykernel import kernelapp as app


<xarray.DataArray (k: 512, l: 512)>
array([[-4.177989, -4.313072, -4.443979, ..., -3.807599, -3.890817, -4.039222],
       [-4.125089, -4.241972, -4.370782, ..., -3.871156, -3.944034, -4.013669],
       [-4.142801, -4.228413, -4.358481, ..., -3.851016, -3.914324, -4.033969],
       ..., 
       [-4.075208, -4.233799, -4.405788, ..., -3.646007, -3.757123, -3.932068],
       [-4.144576, -4.304499, -4.451992, ..., -3.730878, -3.816756, -3.969261],
       [-4.234254, -4.352095, -4.497717, ..., -3.754923, -3.852816, -4.047693]])
Coordinates:
  * k        (k) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * l        (l) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...

In [4]:
iso_f = xf.isotropic_powerspectrum(theta, remove_mean=True, 
                  density=True)

NameError: name 'nbins' is not defined

In [ ]:
iso_f

In [ ]:
fig = plt.figure(figsize=(11,4))

ax1 = fig.add_subplot(121)
ax1.loglog(iso_f.freq_r,iso_f)

In [ ]:
y_fit, a, b = xf.fit_loglog(iso_f.freq_r.values[4:],
                            iso_f.values[4:])

In [ ]:
a